# Basics of Dynamics Systems and Bifurcation Theory
# Workshop \#1, 21 Jan. 2020

#### Navid C. Constantinou, RSES, ANU, 2020

## All dynamical systems are of “first order” 

Any *autonomous* dynamical system can be expressed in the form

$$
 \dot{\phi} = \mathcal{F}(\phi). \tag{1}
$$

Above, $\phi$ includes all variables needed to describe the state of the system. Dot denotes time-derivative, $\dot{\phi}\equiv\mathrm{d}\phi/\mathrm{d}t$.

---

#### Example: the pendulum

For a pendulum that evolves freely in three dimensions, $\phi$ includes the position and the velocity of its free end. In this case, $\phi$ has six components:
$$
  \phi = [x(t), y(t), z(t), \dot{x}(t), \dot{y}(t), \dot{z}(t)].
$$

---


<!-- For the motion of a string on a guitar $\phi$ then includes the deviation of each point on the string from rest, as well as the velocity of each points. Thus $\phi$ in this case consists of two **fields** (otherwise functions) that denote the elevation and velocity of each point of the string. -->

Equation (1) describes the dynamical evolution of our system. Given (1) and the state of the system at some instance $t=t_0$, e.g., $\phi(t_0)=\phi_0$, we can predict the state of the system $\phi(t)$ for all times $t$.



## All dynamical systems are of “first order” 


Any $n$-th order differential equation can be rewritten as a system of $n$ differential equations all being **first order**. In that sense, *all* dynamical systems are first-order in time (but of course their dimensionality may vary).


#### Exercise 1

Show that any **non-autonomous** system of $n$ differential equations corresponds to an **autonomous** system of $n+1$ differential equations.

*Hint*: You can start by an example: Show that the forced 1D harmonic oscillator

$$
  \ddot{x} + 2\gamma\dot{x}+\omega^2 x = f_0\cos(2\pi t),
$$

can be rewritten as a system of 3 equations that **do not** depend explicitly on time. Then generalise this to what the exercise wants.

#### Exercise 2 (optional)

Show that any **nonlinear dynamical system** can be rewritten as a **linear** dynamical system of higher dimensionality.

Take as an example the system:
$$
\dot{x} = -x^2.
$$

How big is the dimension of the equivalent linear system for the example above?

Food for thought: If any nonlinear system can be transformed into a linear one then what's the point of distinguish between *linear* and *nonlinear* dynamical systems?

## Attractors of Dynamical Systems


**Fixed points** or **equilibrium points** are special values of $\phi=\phi^e$ which satisfy:

$$
\mathcal{F}(\phi^e) = 0.  \tag{2}
$$

They are called equilibria or fixed points because if the system finds itself in one of these states then it will stay there for eternity.

The character of a fixed point is studied by assuming small deviations about that fixed point,

$$
  \phi = \phi^e + \phi',\quad\text{where}\quad\|\phi'\|\ll 1.  \tag{3}
$$

Determining how these small deviations will evolve characterises the fixed point. We do that by writing out the linear dynamical system that governs $\phi'$. By inserting Eq. (3) in Eq. (1) and after linearization (i.e., discard of the terms that involve quantites which are quadratic or higher order in components of $\phi'$), we get:

$$
  \dot{\phi'} = \mathcal{F}(\phi^e + \phi') \approx \mathcal{A}(\phi^e)\,\phi',\tag{4}
$$

where $\mathcal{A}$ is a linear operator that depends on the fixed point $\phi^e$.

---

#### Example: the 1D pendulum

<img src="assets/pendulum.png" width=15%>


Consider a pendulum in two dimensions we learn that the angle $\theta$ from rest is governed by
$$
\frac{\mathrm{d}^2\theta}{\mathrm{d}t^2} + \frac{g}{\ell} \sin{\theta} = 0.  \tag{3}
$$
Note that this is a second-order dynamical law. But if we consider $\omega(t)\equiv \mathrm{d}\theta/\mathrm{d}t$ and  $\phi=[ \theta, \omega ]^\mathrm{T}$ then we can rewrite the second-order equation of motion as:
$$
\frac{\mathrm{d}}{\mathrm{d}t} \begin{bmatrix}\theta(t)\\ \omega(t) \end{bmatrix} = \begin{bmatrix}\omega(t)\\-\frac{g}{\ell} \sin[\theta(t)]\end{bmatrix}.  \tag{4}
$$

What are the pendulum's fixed points?
$$
\phi^e_1 = \begin{bmatrix}0\\0\end{bmatrix}\quad\textrm{and}\quad \phi^e_2 = \begin{bmatrix}\pi\\0\end{bmatrix}  \tag{5}
$$

---

\begin{align}
\dot{x} &= u_1 \qquad(u_1=x^2)\\
\dot{u_1} &= 2 x \dot{x} = u_2  \qquad(u_2= 2 x u_1 = 2x^3)\\
\dot{u_2} &= 6x^2 \dot{x} = u_3 \qquad(u_3= 6 x^2 u_1 = 6x^4)\\
&\vdots
\end{align}

### Asymptotic/modal stability


We say that $\phi^e$ is ***attracting*** or ***asymptotically stable*** (or ***modally stable***) if all trajectories that start near $\phi^e$ approach it as $t\to \infty$.

In other words, any trajectory $\phi^e(t)$ that starts within a "distance" $\delta$ of $\phi^e$ is guaranteed to converge to $\phi^e$ ***eventually***.

Asymptotic stability concerns what happens at $t\to\infty$. Eigenanalysis of a dynamical system reveals the $t\to\infty$ behavior of it.

#### Recipe

How do we study the asymptotic/modal stability of these fixed point?

1. We assume a state that slightly perturbed about the fixed point, $\phi = \phi^e + \phi'$.
2. We insert the state in the equation of motion, use that $\phi^2$ satisfies (2), and discard any terms which are products of $\phi'$, e.g. discard $(\phi')^2$, $(\phi')^3$, ...
3. We obtain a linear system of the form
$$
\dot{\phi'} = \mathcal{A}\,\phi'.  \tag{6}
$$
where operator $\mathcal{A}$ depends on the equilibrium state $\phi^e$.
4. Search for eigensolutions $\phi' = \widetilde{\phi}\,e^{\sigma t}$. This transforms (6) in an eigenvalue problem with $\sigma$ the eigenvalue and $\widetilde{\phi}$ the eigenvector/eigenfunction:
$$
\mathcal{A}\,\widetilde{\phi} = \sigma \, \widetilde{\phi}.  \tag{7}
$$
5. Eigenanalysis of $\mathcal{A}$ determines its spectrum (spectrum is the set of all eigenvalues).
6. If we have *at least one eigenvalue* $\sigma$ with $\mathrm{Re}(\sigma)>0$ then we have **modal instability**. Otherwise the equilibrium $\phi^e$ is **modally stable**.


**A bit more on point 6**: We say that if we have an eigenvalue $\sigma$ with $\mathrm{Re}(\sigma)>0$ then we have modal instability. That is because if we have a perturbation which initially looks like the eigenfunction of that eigenfunction, $\phi'(t=0) = \widetilde{\phi}$, then that perturbation will grow away from $\phi^e$ at an exponential rate.

**Exercise**: Confirm the above statement and then generalise it. What happens in the above system if initially we start of with any random perturbation (not with the "special" perturbation $\phi'(t=0) = \widetilde{\phi}$)? Will the system again move away from $\phi^e$ at an exponential rate?

---
#### Example: the pendulum (continued)
For our pendulum example we have that for $\phi^e_1$
$$
 \mathcal{A}_1 = \begin{pmatrix}0 & 1 \\ -\frac{g}{\ell} & 0\end{pmatrix}
$$
while for $\phi^e_2$:
$$
 \mathcal{A}_2 = \begin{pmatrix}0 & 1 \\ +\frac{g}{\ell} & 0\end{pmatrix}
$$

Eigenanalysis of $\mathcal{A}_1$ gives that its eigenvalues are $\sigma = +i\sqrt{\frac{g}{\ell}}, -i\sqrt{\frac{g}{\ell}}$.

Eigenanalysis of $\mathcal{A}_2$ gives that its eigenvalues are $\sigma = +\sqrt{\frac{g}{\ell}}, -\sqrt{\frac{g}{\ell}}$.

**None** of the eigenvalues of $\mathcal{A}_1$ have positive real part $\Rightarrow$ $\phi^e_1$ is **modally stable**.

There is exist one eigenvalue of $\mathcal{A}_2$ with $\mathrm{Re}(\sigma)>0$ $\Rightarrow$ $\phi^e_2$ is **modally unstable**.

**Exercise**: Fill in the missing dots in the pendulum example. First make sure that you can reproduce the above. Then proceed finding the eigenvectors that correspond to those eigenvalues. How would the fastest growing mode look like? 

---

## A geometric look on dynamical systems

A take-away message from lectures is that actually obtaining analytic solutions for a dynamical system is tedious (sometimes even impossible) and, furthermore, it provides very little insight.

Instead, one can get a very good idea of how the dynamical system will behave by drawing the so-called phase portrait.

The phase portrait is constructed by finding attractors (e.g., fixed points or limit cycles) and the corresponding stability characteristic of these attractors. Oftentimes it helps to draw the nullclines (locus of points where each component of $\dot{\phi}$ vanishes).

## Give an example on drawing a phase portrait? E.g., Lotka-Voltera

#### Exercise 3

Plot the phase portraits of the following linear dynamical systems:

*(i)* $\dot{x}=y$, $\dot{y}=-2x-3y$,

*(ii)* $\dot{x}=3x-4y$, $\dot{y}=x-y$,

*(iii)* $\dot{x}=5x+2y$, $\dot{y}=-17x-5y$,

*(iv)* $\dot{x}=4x-3y$, $\dot{y}=8x-6y$,

*(v)* $\dot{x}=5x+10y$, $\dot{y}=-x-y$,

*(vi)* $\dot{x}=-3x+2y$, $\dot{y}=x-2y$,

*(vii)* $\dot{x}=-3x+4y$, $\dot{y}=-2x+3y$,

*(viii)* $\dot{x}=y$, $\dot{y}=-x-2y$.

#### Exercise 4

The damped 1D harmonic oscilator evolves following
$$
 m\ddot{x} + b \dot{x} + k x = 0,
$$
where $b>0$.

Classify the fixed point at the origin and sketch the phase portrait. Investigate how the classification and the phase portrait chagnes as parameters vary.

Hint: Before entering elaborate investigations think how many parameters does the problem *really* depends on? Is it really three?

#### Exercise 5

The Van der Poll oscillator is 
$$
 \ddot{x} + \mu(x^2-1)\dot{x} + \omega^2 x = 0,
$$
where $\mu>0$.

This is like the simple harmonic oscillator but with a nonlinear damping coefficient: $\mu(x^2-1)$ instead of the usual $b/m$. For $|x|>1$ this is like normal damping, but for $|x|<1$ this term is *anti-damping*.

Let's try to build intuition on how this system will behave.

(i) First, study the stability character of the fixed point at the origin.

(ii) For large |x| the equation should behave similarly to a simple harmonic oscillator. That is we expect that the amplitude of the oscillations will decrease. Show that when $|x|\gg1$ the energy:

$$
  E = \tfrac1{2}(x^2+\omega^2\dot{x}^2)
$$

decreases monotonically with time.

(iii) Combine (i) and (ii) and deduce that the system must have a attractor. Is it a fixed point or a limit cycle? For the case of $\mu\ll1$ can you find the attractor?

### References

You can read about stability in general dynamical system in many textbooks. My favorite is:

- Strogatz, S. H. Nonlinear dynamics and chaos: with applications to physics, biology, chemistry, and engineering. CRC Press, 2018.